In [1]:
import spacy

In [2]:
def get_nouns(doc):
    nouns = []
    for token in doc:
        if token.pos_ == 'NOUN':
            tag = token.tag_
            parts = tag.split('|')
            if len(parts) > 3:
                number = parts[3]
                nouns.append([token, number])
            else:
                nouns.append([token, 'S'])
    
    return nouns

def get_nmodifiers(head, doc):
    modifiers = []
    for token in doc:
        if token.head == head:
            if token.pos_ == 'ADJ':
                modifiers.append(token.text.lower())
    return modifiers

In [3]:
s = "O menino foi bem nas provas difíceis e a menina colou a resposta errada."
nlp = spacy.load('pt_core_news_sm')
doc = nlp(s)
for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_, token.head)

nouns = get_nouns(doc)
print(nouns)

for n in nouns:
    print(n, get_nmodifiers(n[0], doc))

O DET <artd>|ART|M|S|@>N det menino
menino NOUN <np-def>|N|M|S|@SUBJ> nsubj nas
foi VERB <mv>|V|PS|3S|IND|@FS-STA cop nas
bem ADV <quant>|ADV|@<ADVL advmod nas
nas ADV ADV|@>N ROOT nas
provas NOUN <first-cjt>|<np-idf>|N|F|P|@<SC obj nas
difíceis ADJ ADJ|F|P|@N< amod provas
e CCONJ <co-fcl>|<co-fmc>|<co-vfin>|KC|@CO cc colou
a DET <artd>|ART|F|S|@>N det menina
menina NOUN <np-def>|N|F|S|@SUBJ> nsubj colou
colou VERB <mv>|V|PS|3S|IND|@FS-STA conj nas
a DET <artd>|ART|F|S|@>N det resposta
resposta NOUN <np-def>|N|F|S|@<ACC obj colou
errada VERB <mv>|V|PCP|F|S|@ICL-N< acl resposta
. PUNCT PU|@PU punct nas
[[menino, 'S'], [provas, 'F'], [menina, 'S'], [resposta, 'S']]
[menino, 'S'] []
[provas, 'F'] ['difíceis']
[menina, 'S'] []
[resposta, 'S'] []


# Utilizar o córpus obtido para P1
- Obter os núcleos (head) dos sintagmas nominais (SN)
- Obter os modificadores do núcleo do SN
- Para os núcleos dos sintagmas nominais, e os seus modificadores
- Criar um dicionário:
``` 
{"núcleo": [("modificador1", freq), ("modificador2", freq)]}
```
- Produzir frases, com o seguinte padrão:
```
"núcleo" verbo_ser "modificador" mais frequente.
```
- o verbo ser, deve ser conjugado como "é" para núcleos de sintagmas nominais no singular e como "são" para núcleos de sintagmas nominais no plural

In [4]:
from nlputils.lexical import Preprocessing
import os

normalizer = Preprocessing()

In [5]:
texts = os.listdir('data/corpora/tecnologia/')
sentences = []
for text in texts:
    with open('data/corpora/tecnologia/' + text, 'r') as text_file:
        lines = text_file.readlines()
        for line in lines:
            sents = normalizer.tokenize_sentences(line)
            sentences.extend(sents)

In [6]:
len(texts), len(sentences)

(2539, 38951)

In [7]:
nouns_and_modifiers = {}

for i, sentence in enumerate(sentences[:400]):
    if i % 100 == 0:
        print("{} of {}".format(i, len(sentences)))
    
    doc = nlp(sentence)
    nouns = get_nouns(doc)
    for n, number in nouns:
        hash_ = '{}-{}'.format(n, number)
        mods = get_nmodifiers(n, doc)
        if len(mods) > 0:
            if hash_ in nouns_and_modifiers:
                modifiers = nouns_and_modifiers[hash_]
                for mod in mods:
                    modifiers.append(mod)
                nouns_and_modifiers[hash_] = modifiers
            else:
                nouns_and_modifiers[hash_] = []
                for mod in mods:
                    nouns_and_modifiers[hash_].append(mod)

0 of 38951
100 of 38951
200 of 38951
300 of 38951


In [8]:
from collections import Counter
nouns_and_modifiers2 = {}
for hash_, modifiers in nouns_and_modifiers.items():
    modifiers = Counter(modifiers)
    nouns_and_modifiers2[hash_] = modifiers

In [11]:
for hash_, counter in nouns_and_modifiers2.items():
    noun, number = hash_.split("-")
    verb = 'é'
    if number == 'P':
        verb = 'são'
    modifier = counter.most_common(1)
    print("{} {} {}".format(noun, verb, modifier[0][0]))        

inteligência é artificial
bloco é econômico
atividades são diversas
redes são sociais
comportamento é inteligente
metas são específicas
soluções são confiáveis
normas é administrativas
orientações são gerais
relevância é diretrizes
seres são humanos
objetos são técnicos
orientação é complementar
representações é humanas
orientação é sexual
inconsistências são eventuais
pessoas são brancas
negros são na
sociedades são equitativas
iniciativa é humana
autonomia é humana
seguros é fiáveis
controle é pleno
evolução é social
responsabilidade é ecológica
dimensão é ética
vantagem é competitiva
academiaEm é dúvida
TechTudo24/04/2017 é 07h00
pulseiras são inteligentes
recursos são inúmeros
treino é próprio
rastreador é fitness
tradução é livre
frequências é cardíacas
opção é melhor
tênis é yoga
recursos é disponíveis
tecnologias é específicas
ponto é chave
método é melhor
dados são precisos
intensidade é alta
contato é físico
design é discreto
pulseira é relógio
aparelho é compatível
forma é gr